# Datenauswahl und Verarbeitung

In [1]:
# SP_AP_DataScience

import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler # Import bleibt, falls später benötigt
import os
import traceback

# --- Konstanten und Konfiguration ---
OUTPUT_FOLDER = "financial_data_prepared" # Ordner für aufbereitete Daten & Plots
RAW_DATA_FOLDER = "financial_data_raw"    # Ordner für optional gespeicherte Rohdaten
SAVE_RAW_DATA = True                      # Rohdaten separat speichern?

# Definition der zu analysierenden Ticker (reduziert)
TICKERS = {
    "Stocks":  ["GOOG"],  # Ursprünglich: ["AAPL", "MSFT", "GOOG"]
    "ETFs":    ["ACWI"],  # Ursprünglich: ["ACWI", "SPY"]
    "Indices": ["^IXIC"], # Ursprünglich: ["^GSPC", "^DJI", "^IXIC"]
}
START_DATE = "2020-01-01"
END_DATE = "2025-05-12"     # Festes Enddatum
PLOT_FIGSIZE = (12, 6)      # Standardgröße für Plots

# --- Hilfsfunktionen ---

def save_single_plot(fig, ax, output_folder, base_filename_part, plot_name_suffix):
    """Speichert eine Matplotlib Figure als PNG und schließt sie."""
    try:
        ax.grid(True, alpha=0.3); fig.tight_layout()
        plot_filename = os.path.join(output_folder, f"{base_filename_part}_{plot_name_suffix}.png")
        fig.savefig(plot_filename)
    except Exception as e: print(f"    FEHLER beim Speichern des Plots {plot_filename}: {e}")
    finally: plt.close(fig)

# --- Hauptverarbeitungsfunktion pro Ticker ---

def download_prepare_visualize(ticker, asset_type, start_date, end_date, output_folder, raw_data_folder=None, save_raw=False):
    """Lädt, bereinigt (OHNE Volumen), berechnet Indikatoren manuell, visualisiert & speichert Daten."""
    print(f"\n--- Verarbeite: {ticker} ({asset_type}) ---")
    safe_ticker_name = ticker.replace('^', '')
    prepared_data = None
    data = None

    try:
        # 1. Download
        print(f"  1. Download: {start_date} bis {end_date}")
        data = yf.download(ticker, start=start_date, end=end_date, progress=False)

        if data.empty: print(f"    FEHLER: Keine Daten für {ticker} gefunden."); return None
        print(f"    Info: {len(data)} Datenpunkte erhalten.")

        # MultiIndex-Behandlung
        if isinstance(data.columns, pd.MultiIndex):
            print(f"    Info: MultiIndex-Spalten erkannt: {data.columns.names}. Vereinfache...")
            try:
                if 'Ticker' in data.columns.names: data.columns = data.columns.droplevel('Ticker')
                else: data.columns = data.columns.droplevel(1)
            except Exception as e_droplevel: print(f"    FEHLER: MultiIndex-Vereinfachung fehlgeschlagen: {e_droplevel}"); return None

        # 2. Rohdaten optional speichern
        if save_raw and raw_data_folder:
            os.makedirs(raw_data_folder, exist_ok=True)
            raw_filename = f"{safe_ticker_name}_{asset_type.lower()}_raw_data.csv"
            raw_filepath = os.path.join(raw_data_folder, raw_filename)
            try: data.reset_index().to_csv(raw_filepath, index=False)
            except Exception as e: print(f"    Warnung: Fehler beim Speichern der Rohdaten: {e}")

        # 3. Bereinigung & Validierung (ohne Volumen)
        print("  2. Bereinigung & Validierung (ohne Volumen)")
        required_cols = ['Open', 'High', 'Low', 'Close']
        if 'Adj Close' in data.columns: required_cols.append('Adj Close')
        elif 'Adjusted Close' in data.columns:
             data.rename(columns={'Adjusted Close': 'Adj Close'}, inplace=True); required_cols.append('Adj Close')

        missing_cols = [col for col in required_cols if col not in data.columns]
        if missing_cols: print(f"    FEHLER: Fehlende Preis-Spalten: {missing_cols}. Spalten: {data.columns}"); return None

        # Standard Numerische Konvertierung
        print("    Info: Standard numerische Konvertierung (Preisdaten)...")
        for col in required_cols:
            try: data[col] = pd.to_numeric(data[col], errors='coerce')
            except Exception as e: print(f"    FEHLER bei Konvertierung von '{col}': {e}"); return None
            if not pd.api.types.is_numeric_dtype(data[col]): print(f"    FEHLER: '{col}' nicht numerisch (Typ: {data[col].dtype})."); return None

        # 4. Feature Engineering: Technische Indikatoren (manuell)
        print("  3. Feature Engineering (manuell)")
        price_col = 'Adj Close' if 'Adj Close' in data.columns else 'Close'
        print(f"    Info: Verwende '{price_col}' für Indikatoren.")
        if price_col not in data.columns: print(f"   FEHLER: Preis-Spalte '{price_col}' fehlt."); return None

        # --- Manuelle Indikatorberechnungen ---
        data["SMA_20"] = data[price_col].rolling(window=20).mean()
        data["SMA_50"] = data[price_col].rolling(window=50).mean()
        data["SMA_200"] = data[price_col].rolling(window=200).mean()
        data['StdDev_20'] = data[price_col].rolling(window=20).std()
        data['Upper'] = data['SMA_20'] + 2 * data['StdDev_20']
        data['Lower'] = data['SMA_20'] - 2 * data['StdDev_20']
        delta = data[price_col].diff()
        up, down = delta.clip(lower=0), -1*delta.clip(upper=0)
        ema_up, ema_down = up.ewm(com=13, adjust=False).mean(), down.ewm(com=13, adjust=False).mean()
        rs = ema_up / ema_down.replace(0, np.nan)
        data['RSI'] = 100 - (100 / (1 + rs))
        data['RSI'] = data['RSI'].replace([np.inf, -np.inf], 100).fillna(50)
        data['EMA_12'] = data[price_col].ewm(span=12, adjust=False).mean()
        data['EMA_26'] = data[price_col].ewm(span=26, adjust=False).mean()
        data['MACD'] = data['EMA_12'] - data['EMA_26']
        data['Signal'] = data['MACD'].ewm(span=9, adjust=False).mean()
        data['MACD_Histogram'] = data['MACD'] - data['Signal']
        # --- Ende Indikatorberechnungen ---

        # 5. Finale NaN-Bereinigung
        print("  4. Finale NaN-Bereinigung")
        initial_rows = len(data)
        data.dropna(inplace=True)
        rows_dropped = initial_rows - len(data)
        if rows_dropped > 0 : print(f"    Info: {rows_dropped} Zeilen mit NaN entfernt.")
        if data.empty: print(f"    FEHLER: Keine Daten nach NaN-Bereinigung."); return None

        # 6. Visualisierung (ohne Volumen)
        print("  5. Visualisierung (ohne Volumen)")
        base_filename_part = f"plot_{safe_ticker_name}_{asset_type.lower()}"
        # --- Plotting-Code ---
        # Plot: Bollinger Bänder
        if all(c in data.columns for c in [price_col, "SMA_20", "Upper", "Lower"]):
            fig, ax = plt.subplots(figsize=PLOT_FIGSIZE); ax.plot(data.index, data[price_col], label=f"{price_col} Kurs", lw=1); ax.plot(data.index, data["Upper"], label="Oberes Band", c='red', ls='--', lw=0.8); ax.plot(data.index, data["Lower"], label="Unteres Band", c='green', ls='--', lw=0.8); ax.plot(data.index, data["SMA_20"], label="SMA 20", c='orange', ls=':', lw=0.8); ax.fill_between(data.index, data['Lower'], data['Upper'], color='grey', alpha=0.1); ax.set_title(f"Bollinger Bänder: {ticker} (Basis: {price_col})"); ax.legend(); save_single_plot(fig, ax, output_folder, base_filename_part, "BollingerBands")
        # Plot: RSI
        if "RSI" in data.columns:
            fig, ax = plt.subplots(figsize=PLOT_FIGSIZE); ax.plot(data.index, data["RSI"], label="RSI", color='purple', lw=1); ax.axhline(70, color='red', ls='--', label='Überkauft (70)', lw=0.8); ax.axhline(30, color='green', ls='--', label='Überverkauft (30)', lw=0.8); ax.set_title(f"RSI: {ticker}"); ax.set_ylim(0, 100); ax.legend(); save_single_plot(fig, ax, output_folder, base_filename_part, "RSI")
        # Plot: MACD
        if all(c in data.columns for c in ["MACD", "Signal", "MACD_Histogram"]):
             fig, ax = plt.subplots(figsize=PLOT_FIGSIZE); hist_colors = ['green' if v >= 0 else 'red' for v in data['MACD_Histogram']]; ax.plot(data.index, data["MACD"], label="MACD", color='blue', lw=1); ax.plot(data.index, data["Signal"], label="Signal", color='red', ls='--', lw=1); ax.bar(data.index, data['MACD_Histogram'], label='Histogramm', color=hist_colors, alpha=0.6, width=1.0); ax.axhline(0, color='grey', ls='-', lw=0.5); ax.set_title(f"MACD: {ticker}"); ax.legend(); save_single_plot(fig, ax, output_folder, base_filename_part, "MACD")
        # Plot: Kurs & SMAs
        if price_col in data.columns:
            fig, ax = plt.subplots(figsize=PLOT_FIGSIZE)
            ax.plot(data.index, data[price_col], label=f"{price_col} Kurs", lw=1.5)
            if "SMA_50" in data.columns:
                ax.plot(data.index, data["SMA_50"], label="SMA 50", c='orange', ls='--', lw=1)
            if "SMA_200" in data.columns:
                ax.plot(data.index, data["SMA_200"], label="SMA 200", c='m', ls=':', lw=1)
            ax.set_title(f"Kurs & SMAs: {ticker} (Basis: {price_col})"); ax.legend(); save_single_plot(fig, ax, output_folder, base_filename_part, "Price_SMA")
        # --- Ende Plotting-Code ---

        # 7. Aufbereitete Daten speichern
        print("  6. Speichern der aufbereiteten Daten")
        # 'Volume'-Spalte entfernen, falls vorhanden
        if 'Volume' in data.columns:
            data = data.drop(columns=['Volume'])

        prepared_filename = f"prepared_{safe_ticker_name}_{asset_type.lower()}_data.csv"
        output_path = os.path.join(output_folder, prepared_filename)
        try:
            data.to_csv(output_path)
            print(f"    Aufbereitete Daten gespeichert: {output_path}")
            prepared_data = data # Wird für Excel benötigt
        except Exception as e: print(f"    FEHLER beim Speichern der aufbereiteten CSV: {e}")

    # Fehlerbehandlung
    except KeyError as e:
        print(f"  FEHLER (KeyError/Spalte nicht gefunden) für {ticker}: {e}.")
        print(f"    Debug: Verfügbare Spalten waren: {data.columns if data is not None else 'Keine Daten geladen'}")
    except Exception as e:
        print(f"  FEHLER (Allgemein) für {ticker}: {e}")
        print(traceback.format_exc())

    return prepared_data # Gibt das DataFrame zurück (für Excel)

# --- Hauptausführung ---
if __name__ == "__main__":
    print("=== Starte Finanzdaten-Verarbeitungsskript ===")
    os.makedirs(OUTPUT_FOLDER, exist_ok=True)
    if SAVE_RAW_DATA: os.makedirs(RAW_DATA_FOLDER, exist_ok=True)
    # all_prepared_data Dictionary wird WIEDER benötigt für Excel-Export
    all_prepared_data = {}

    # Iteration durch Ticker
    for asset_type, ticker_list in TICKERS.items():
        print(f"\n>>> Verarbeite Anlageklasse: {asset_type} <<<")
        for ticker in ticker_list:
            processed_df = download_prepare_visualize(
                ticker, asset_type, START_DATE, END_DATE,
                OUTPUT_FOLDER, RAW_DATA_FOLDER if SAVE_RAW_DATA else None, SAVE_RAW_DATA
            )
            # Sammle erfolgreiche Ergebnisse für Excel-Export
            if processed_df is not None and not processed_df.empty:
                all_prepared_data[f"{ticker}_{asset_type}"] = processed_df

    # Optional: Excel-Export
    excel_filename = os.path.join(OUTPUT_FOLDER, "alle_vorbereiteten_daten_final.xlsx")
    if all_prepared_data:
        print(f"\nSchreibe {len(all_prepared_data)} Ergebnisse nach: {excel_filename}")
        try:
            # benötigt 'openpyxl': pip install openpyxl
            with pd.ExcelWriter(excel_filename) as writer:
                for sheet_name, df in all_prepared_data.items():
                    safe_sheet_name = sheet_name.replace('^', '').replace(':', '').replace('/', '').replace('\\', '').replace('*', '').replace('?', '').replace('[', '').replace(']', '')[:31]
                    df.to_excel(writer, sheet_name=safe_sheet_name)
            print("Excel-Datei erfolgreich geschrieben.")
        except Exception as e: print(f"FEHLER beim Schreiben der Excel-Datei: {e}")
    else: print("\nKeine Daten erfolgreich verarbeitet oder gesammelt, keine Excel-Datei erstellt.")


    print("\n=== Skriptausführung beendet ===")

=== Starte Finanzdaten-Verarbeitungsskript ===

>>> Verarbeite Anlageklasse: Stocks <<<

--- Verarbeite: GOOG (Stocks) ---
  1. Download: 2020-01-01 bis 2025-05-12
YF.download() has changed argument auto_adjust default to True
    Info: 1346 Datenpunkte erhalten.
    Info: MultiIndex-Spalten erkannt: ['Price', 'Ticker']. Vereinfache...
  2. Bereinigung & Validierung (ohne Volumen)
    Info: Standard numerische Konvertierung (Preisdaten)...
  3. Feature Engineering (manuell)
    Info: Verwende 'Close' für Indikatoren.
  4. Finale NaN-Bereinigung
    Info: 199 Zeilen mit NaN entfernt.
  5. Visualisierung (ohne Volumen)
  6. Speichern der aufbereiteten Daten
    Aufbereitete Daten gespeichert: financial_data_prepared\prepared_GOOG_stocks_data.csv

>>> Verarbeite Anlageklasse: ETFs <<<

--- Verarbeite: ACWI (ETFs) ---
  1. Download: 2020-01-01 bis 2025-05-12
    Info: 1346 Datenpunkte erhalten.
    Info: MultiIndex-Spalten erkannt: ['Price', 'Ticker']. Vereinfache...
  2. Bereinigung & Val